In [26]:
import openai
from dataset.persent import PerSenTDataset
from os.path import join, isfile
import pandas as pd
from tqdm import tqdm

In [27]:
# openai.api_key = 

In [28]:
datadir = "../data/PerSenT"
train_filepath = "train.csv"
dataset = PerSenTDataset(join(datadir,train_filepath))
len(dataset)

3355

In [29]:
x, y = dataset[6]
x, y

('Lubna Hussein was among 13 women arrested July 3 in a raid by the public order police in Khartoum. Ten of the women were fined and flogged two days later. But Hussein and two others decided to go to trial.\n Hussein said Friday she would rather go to jail than pay any fine  out of protest of the nation\'s strict laws on women\'s dress.\n The case has made headlines in Sudan and around the world and Hussein used it to rally world opinion against the country\'s morality laws based on a strict interpretation of Islam.\n Galal al-Sayed  Hussein\'s lawyer  said he advised her to pay the fine before appealing the decision. She refused  he said  "She insisted."\n As a U.N. staffer  Hussein should have immunity from prosecution but she has opted to resign so that she could stand trial and draw attention to the case.\n In a column published in the British daily the Guardian Friday  Hussein said her case is not an isolated one  but is a showcase of repressive laws in a country with a long hist

In [30]:

template_one_paraphrase = "Generate a paraphrase for the following text, preserving the sentiment of the following statement: {text}"
template_one_more_words = "Generate another paraphrase by changing more words also keeping the sentiment"

template_3_paraphrases = "Generate 3 different paraphrases for the following text, preserving the sentiment of the following statement: {text}"

template_new_text = "Based on the given text, generate another text with a completely new theme, but be inspired by the original text and keep the sentiment of the old one in the new text. Original text: {text}"
template_new_text_v2 = "Based on the given text, generate another text with a completely new theme, but be inspired by the original text and keep a {sentiment} sentiment. Original text: {text}" 

# chat = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo", messages=[{"role": "user", "content": "tell something"}]
# )
# chat

In [31]:
# chat.choices[0].message.content

In [32]:
def save_df(i, dictionary, filename):
    df = pd.DataFrame(data=dictionary)
    if i == 0:
        df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode="a", index=False, header=False)

def get_response(messages):
    not_done = True
    too_long = False
    curr_response = ""
    while not_done:
        try:
            chat = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", messages=messages
            )
            curr_response = chat.choices[0].message.content
            not_done = False
        except openai.error.InvalidRequestError:
            too_long = True
            not_done = False
        except (openai.error.ServiceUnavailableError, openai.error.APIError):
            time.sleep(5)
    curr_response = curr_response.replace("\n", " ")
    return curr_response, too_long

def prepare_first_message(template, x, y):
    form = {"text": x}
    if "{sentiment}" in template:
        form["sentiment"] = y
    curr_content = template.format(**form)
    message = [{"role": "user", "content": curr_content}]
    return message

def prepare_second_message(template, msg, first_response):
    form = {}
    if "{text}" in template:
        form["text"] = x
    if "{sentiment}" in template:
        form["sentiment"] = y
    curr_content = template.format(**form)
    msg.append({"role": "assistant", "content": first_response})
    msg.append({"role": "user", "content": curr_content})
    return msg


def create_one_augumentation(dataset, template, filename, threshold=10):
    augumented_data = {"text": [], "label": []}
    # messeges = [{"role": "user", "content": "{content}"}]
    if isfile(filename):
        saved = pd.read_csv(filename)
        saved_examples = saved.shape[0]
    else:
        saved_examples = 0
    for i, (x, y) in tqdm(list(enumerate(zip(dataset.X, dataset.Y)))):
        if i < saved_examples:
            continue
        curr_msg = prepare_first_message(template, x, y)
        curr_response, too_long = get_response(curr_msg)
        if too_long:
            continue
        augumented_data["text"].append(curr_response)
        augumented_data["label"].append(y)
        if i % threshold == 0:
            save_df(i, augumented_data, filename)
            augumented_data = {"text": [], "label": []}
    if len(augumented_data["text"]) > 0:
        save_df(-1, augumented_data, filename)
    return filename

def prepare_collections(filename, suffix):
    final_filename = filename.replace(".csv", f"_{suffix}.csv")
    data = {"text": [], "label": []}
    if isfile(final_filename):
        saved = pd.read_csv(final_filename)
        saved_examples = saved.shape[0]
    else:
        saved_examples = 0
    return final_filename, data, saved_examples

def create_many_augumentations(dataset, templates, filename, threshold=10):
    first_filename, first_data, first_saved = prepare_collections(filename, "normal")
    second_filename, second_data, second_saved = prepare_collections(filename, "different_words")
    assert first_saved == second_saved

    for i, (x, y) in tqdm(list(enumerate(zip(dataset.X, dataset.Y)))):
        if i < first_saved:
            continue
        first_msg = prepare_first_message(templates[0], x, y)
        first_response, too_long = get_response(first_msg)
        if too_long:
            continue
        first_data["text"].append(first_response)
        first_data["label"].append(y)

        second_msg = prepare_second_message(templates[1], first_msg, first_response)
        second_response, _ = get_response(second_msg)

        second_data["text"].append(second_response)
        second_data["label"].append(y)

        if i % threshold == 0:
            save_df(i, first_data, first_filename)
            save_df(i, second_data, second_filename)
            first_data = {"text": [], "label": []}
            second_data = {"text": [], "label": []}
        break
    if len(first_data["text"]) > 0:
        save_df(-1, first_data, first_filename)
    if len(second_data["text"]) > 0:
        save_df(-1, second_data, second_filename)
    return first_filename, second_filename
        

In [33]:
filename = join(datadir, "paraphrase_train.csv")
fn1, fn2 = create_many_augumentations(dataset, (template_one_paraphrase, template_one_more_words), filename, threshold=10)

  0%|                                                                              | 1/3355 [00:16<15:43:27, 16.88s/it]


In [34]:
df1 = pd.read_csv(fn1)
df2 = pd.read_csv(fn2)

In [40]:
df1

,text,label
0,"As an AI, I don't have feelings, but I'm here ...",Negative
1,The individual responsible for taking hostages...,Neutral


In [41]:
df2

,text,label
0,"As an AI, I don't have feelings or emotions, b...",Negative
1,The former Police Senior Inspector Rolando Men...,Neutral


In [12]:
# filename = join(datadir, "new_text_train.csv")

# create_one_augumentation(dataset, template_new_text, filename, threshold=100)

In [29]:
# df = pd.read_csv(filename)

In [30]:
# df

,text,label
0,"Based on the given text, generate another text...",Negative
1,"Based on the given text, generate another text...",Neutral
2,"Based on the given text, generate another text...",Negative
3,"Based on the given text, generate another text...",Neutral
4,"Based on the given text, generate another text...",Neutral
...,...,...
3350,"Based on the given text, generate another text...",Neutral
3351,"Based on the given text, generate another text...",Neutral
3352,"Based on the given text, generate another text...",Negative
3353,"Based on the given text, generate another text...",Positive


In [3]:
# curr_msg = [{"role":"user", "content": "How are you?"}]
# chat = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo", messages=curr_msg
# )
# curr_response = chat.choices[0].message.content
# curr_response

"As an AI, I don't have feelings, but I'm here to help. How can I assist you today?"

In [5]:
# curr_msg2 = [{"role":"user", "content": "How are you?"},
#             {"role": "assistant", "content": curr_response},
#             {"role":"user", "content": "Tell me what is blue and smell like red paint?"}]
# chat2 = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo", messages=curr_msg2
# )
# curr_response2 = chat2.choices[0].message.content
# curr_response2

'There is no object or substance that is inherently blue and smells like red paint, as the color blue and the smell of red paint are distinct qualities.'

In [6]:
# chat2.choices

[<OpenAIObject at 0x7fad02ca3cb0> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "There is no object or substance that is inherently blue and smells like red paint, as the color blue and the smell of red paint are distinct qualities."
   },
   "finish_reason": "stop"
 }]